<a href="https://colab.research.google.com/github/amien1410/kaggle-projects/blob/main/Amazon_Discount_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install Kaggle Modules to download the dataset
!pip install kaggle

# Load Kaggle user authorization key
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive/kaggle'

# Download the dataset, get the user and dataset name from the url "https://www.kaggle.com/datasets/shivamb/netflix-shows"
!kaggle datasets download -d lokeshparab/amazon-products-dataset

# Unzip the dataset
!unzip -q "/content/amazon-products-dataset"

Dataset URL: https://www.kaggle.com/datasets/zusmani/uberdrives
License(s): DbCL-1.0
  0% 0.00/16.9k [00:00<?, ?B/s]
100% 16.9k/16.9k [00:00<00:00, 31.9MB/s]
